# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [742]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [743]:
import numpy as np

from itertools import combinations
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [744]:


def build_classifiers():
    candidate_classes = [
        LinearRegression,
        KNeighborsClassifier,
        SVC,
        DecisionTreeClassifier,
        GaussianNB,
        QuadraticDiscriminantAnalysis
    ]

    best_combo = None
    best_acc   = -np.inf

    for idx, trio in enumerate(combinations(candidate_classes, 3), 1):
        clfs = [cls() for cls in trio]
        for clf in clfs:
            clf.fit(data_set, labels)

        preds = build_stacked_classifier(clfs)
        acc = accuracy_score(test_labels, preds)

        names = [c.__name__ for c in trio]
        print(f" {idx:2d}) {names[0]:<25} + {names[1]:<25} + {names[2]:<25}         - acc = {acc:.4f}")

        if acc > best_acc:
            best_acc   = acc
            best_combo = clfs

    best_names = [clf.__class__.__name__ for clf in best_combo]
    print("\nBest trio found:")
    print(f"        - {best_names[0]}, {best_names[1]}, {best_names[2]}")
    print(f"        - Stacked accuracy = {best_acc:.4f}\n")

    return best_combo


In [745]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = LogisticRegression()
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [746]:
classifiers = build_classifiers()
predicted = build_stacked_classifier(classifiers)
accuracy = accuracy_score(test_labels, predicted)
print(accuracy)

  1) LinearRegression          + KNeighborsClassifier      + SVC                               - acc = 0.9500
  2) LinearRegression          + KNeighborsClassifier      + DecisionTreeClassifier            - acc = 0.9500
  3) LinearRegression          + KNeighborsClassifier      + GaussianNB                        - acc = 0.9500
  4) LinearRegression          + KNeighborsClassifier      + QuadraticDiscriminantAnalysis         - acc = 0.9500
  5) LinearRegression          + SVC                       + DecisionTreeClassifier            - acc = 0.9500
  6) LinearRegression          + SVC                       + GaussianNB                        - acc = 0.9000
  7) LinearRegression          + SVC                       + QuadraticDiscriminantAnalysis         - acc = 0.8500
  8) LinearRegression          + DecisionTreeClassifier    + GaussianNB                        - acc = 0.8500
  9) LinearRegression          + DecisionTreeClassifier    + QuadraticDiscriminantAnalysis         - acc = 0.800

## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [747]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

Fill the two functions below:

In [748]:
def set_new_weights(model):
    I = ( model.predict(test_set) != test_labels).astype(int)
    new_w = 1 + I
    return new_w / new_w.sum()

Train the classifier with the code below:

In [749]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00130804 0.00065402 0.00130804 0.00065402 0.00130804 0.00130804
 0.00130804 0.00130804 0.00065402 0.00065402 0.00065402 0.00130804
 0.00065402 0.00065402 0.00130804 0.00130804 0.00130804 0.00130804
 0.00065402 0.00065402 0.00065402 0.00065402 0.00130804 0.00130804
 0.00065402 0.00130804 0.00065402 0.00065402 0.00130804 0.00130804
 0.00065402 0.00130804 0.00130804 0.00065402 0.00130804 0.00130804
 0.00130804 0.00065402 0.00065402 0.00130804 0.00130804 0.00065402
 0.00130804 0.00130804 0.00065402 0.00130804 0.00130804 0.00065402
 0.00065402 0.00065402 0.00130804 0.00130804 0.00130804 0.00065402
 0.00130804 0.00130804 0.00065402 0.00065402 0.00130804 0.00130804
 0.00130804 0.00130804 0.00130804 0.00065402 0.00130804 0.00065402
 0.00130804 0.00065402 0.00130804 0.00130804 0.00065402 0.00130804
 0.00130804 0.00065402 0.00130804 0.00130804 0.00130804 0.00065402
 0.00130804 0.00130804 0.00130804 0.00130804 0.00130804 0.00065402
 0.00130804 0.00065402 0.00130804 0.00130804 0.00130804 0.0013

Set the validation data set:

In [750]:
validate_x, validate_label = generate_data(200, dimension, labels)

Fill the prediction code:

In [751]:
def get_prediction(x):
    preds = np.column_stack([clf.predict(x) for clf in classifiers])
    final = []
    for i in range(preds.shape[0]):
        counts = np.bincount(preds[i, :])
        final.append(np.argmax(counts))
    return np.array(final)

Test it:

In [752]:
prediction = get_prediction(validate_x)

print(prediction)

acc = accuracy_score(validate_label, prediction)
print(f"Accuracy for val: {acc:.3f}")

[0 0 1 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0
 0 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 1 1
 1 0 1 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0
 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0
 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 1 0
 1 0 1 0 1 0 0 1 0 1 0 0 0 0 0]
Accuracy for val: 0.505
